In [1]:
from langchain.chat_models import ChatOpenAI
from kor import create_extraction_chain, Object, Text, Option, Selection

In [16]:
llm = ChatOpenAI(
    base_url="http://localhost:1234/v1",
    api_key="not-needed",
    temperature=0,
    max_tokens=2000,
    streaming=True
)

In [9]:
apps= Object(
    id="apps",
    description="User wants to collection of apps, mostly web or native apps",
    examples=[],
    many=True,
    attributes=[
        Text(
            id="name",
            description="User wants to use this technology",
            examples=[
                ("Technology nextjs as metaframework", "next"),
                ("use reactjs for web", "react"),
                ("use expo for native", "expo"),
            ]
        ),
    ]
)

packages = Object(
    id="packages",
    description="User wants to collection of packages, which help and support apps",
    examples=[],
    many=True,
    attributes=[
        Text(
            id="name",
            description="User wants to use this technology as support packages",
            examples=[
                ("use shadcn ui components as design system in package", "shadcn-ui"),
                ("use tailwind for all projects", "tailwind"),
                ("use trpc as api for all projects", "trpc"),
            ]
        ),
    ]
)

styling = Selection(
    id="styling",
    description="What is prefered styling library",
    options=[
        Option(id="css", description="Css"),
        Option(id="tailwind", description="Tailwind"),
    ],
    examples=[
        ("I prefer tailwind for styling", "tailwind"),
        ("use default styling", "tailwind"),
        ("css is fine for styling", "css"),
    ],
    null_examples=[
        "I like bootstrap",
        "use shadcn",
    ],
    many=False
    
)

schema = Object(
    id="type",
    description="User need a collection of apps and package as turborepo setup for his project",
    attributes=[
        apps,
        packages,
        styling
    ],
)


In [10]:
from kor import TypeScriptDescriptor

In [11]:
descriptor = TypeScriptDescriptor()
print(descriptor.describe(schema))

```TypeScript

type: { // User need a collection of apps and package as turborepo setup for his project
 apps: Array<{ // User wants to collection of apps, mostly web or native apps
  name: string // User wants to use this technology
 }>
 packages: Array<{ // User wants to collection of packages, which help and support apps
  name: string // User wants to use this technology as support packages
 }>
 styling: "css" | "tailwind" // What is prefered styling library
}
```



In [17]:

chain = create_extraction_chain(llm, schema, encoder_or_encoder_class='json')
chain.run("i want turborepo setup with next and expo as native app with tailwind as styling.")['data']

{'type': {'apps': [{'name': 'next'}, {'name': 'expo'}],
  'packages': [{'name': 'shadcn-ui'}, {'name': 'tailwind'}]}}

In [19]:
async for s in chain.astream("i want turborepo setup with next and expo as native app with tailwind as styling."):
    print(s, flush=True)

{'text': {'data': {'type': {'apps': [{'name': 'next'}, {'name': 'expo'}]}}, 'raw': '<json>{"type": {"apps": [{"name": "next"}, {"name": "expo"}]}}</json>\n', 'errors': [], 'validated_data': {}}}
